In [1]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})
import time


# CMPS 6610
# Algorithms

## Recurrences


Recurrences are a way to capture the behavior of recursive algorithms.

Key ingredients: 

- Base case ($n = c$): constant time 
- Inductive case ($n > c$): recurse on smaller instance and use output to compute solution

Actually recursion is a conceptual way to view algorithm execution, and we can reframe an algorithm specification to make it recursive.



In [2]:
def selection_sort(L):
    for i in range(len(L)):
        print(L)
        m = L.index(min(L[i:]))
        L[i], L[m] = L[m], L[i]
    return L
                   
selection_sort([2, 1, 4, 3, 9])

[2, 1, 4, 3, 9]
[1, 2, 4, 3, 9]
[1, 2, 4, 3, 9]
[1, 2, 3, 4, 9]
[1, 2, 3, 4, 9]


[1, 2, 3, 4, 9]

In [3]:
def selection_sort_recursive(L):
    print('L=%s' % L)
    if (len(L) == 1):
        return(L)
    else:
        m = L.index(min(L))
        L[0], L[m] = L[m], L[0]
        return [L[0]] + selection_sort_recursive(L[1:])
    
selection_sort_recursive([2, 1, 999, 4, 3])

L=[2, 1, 999, 4, 3]
L=[2, 999, 4, 3]
L=[999, 4, 3]
L=[4, 999]
L=[999]


[1, 2, 3, 4, 999]

In [4]:
# faster argmin.
L = [2, 1, 999, 4, 3]
m = min(enumerate(L), key=lambda x: x[1])[0]

Are these the same algorithm? Can we give a SPARC specification?

<p><span class="math display">\[\begin{array}{l}  
\mathit{selectionsort}~~L = 
\\  
~~~~~\texttt{if}~|L| = 1~\texttt{then}    
\\  
~~~~~~~~~~~~~\texttt{return}~~L  
\\  
~~~~~~~~~\texttt{else}
\\
~~~~~~~~~~~~\texttt{let}\\
~~~~~~~~~~~~~~~m = \texttt{minimum element in}~~L\\
~~~~~~~~~~~~\texttt{in}\\ 
~~~~~~~~~~~~~~~\texttt{Cons}(m, (\mathit{selectionsort~~\langle x | x\in L~~and~~x\neq m \rangle})) 
\end{array}\]</span></p>



What is the running time and why?

$\begin{eqnarray}
W(n) &=& W(n-1) + n \\
 &=& W(n-2) + 2n-1 \\
&\vdots&
\end{eqnarray}$


$\begin{eqnarray}
W(n) &=& \sum_{i=1}^n i  \\
&=& \frac{n(n+1)}{2} \\
&=& \Theta(n^2).
\end{eqnarray}$


The recurrence for Selection Sort is somewhat simple - what if we have multiple recursive calls and split the input? (This is actually what *divide-and-conquer* algorithms do.)

We'll look at methods to solve recurrences in order to obtain big-O bounds for recursive algorithms.

We will:

- Get intuition for recurrences by looking the recursion tree. 

- Develop the **brick** method to quickly state asymptotic bounds on a recurrence by looking at the shape of the tree.

Let's look at the specification and recurrence for Merge Sort: 

<p><span class="math display">\[\begin{array}{l}  
\mathit{mergeSort}~a =  
\\   
~~~~\texttt{if}~|a| \leq 1~\texttt{then}  
\\   
~~~~~~~~a  
\\  
~~~~\texttt{else}  
\\   
~~~~~~~~\texttt{let}  
\\  
~~~~~~~~~~~~(l,r) = \mathit{splitMid}~a  
\\   
~~~~~~~~~~~~(l',r') = (\mathit{mergeSort}~l \mid\mid{} \mathit{mergeSort}~r)  
\\  
~~~~~~~~\texttt{in}  
\\   
~~~~~~~~~~~~\mathit{merge} (l',r')  
\\  
~~~~~~~~\texttt{end}  
\end{array}\]</span></p>

Suppose that the merging step can be done with $O(n)$ work and $O(\log n)$ span. Then recurrence for the work is: 

$ \begin{equation}
W(n) = \begin{cases}
  c_b, & \text{if $n=1$} \\
  2W(n/2) + c_1n + c_2, & \text{otherwise} 
  \end{cases}
\end{equation}$

How do we solve this recurrence to obtain $W(n) = O(n\log n)$?





![alttext](mergesort_tree.png)

The recursion tree for Merge Sort has linear work at every level except at the leaves. There are a logarithmic number of levels and a linear number of leaves so we obtain an asymptotic bound of $O(n\log n)$ for the work.

## Solving Recurrences with the Tree Method 

<br>


<div>size at level $i$</div> <div style="text-align: right"> cost at level $i$ </div>

![merge-tree.jpg](merge-tree.jpg)

### Recipe: 
1. Expand tree for two levels.
2. Determine the cost of each level $i$ ($i$ starts at $0$).
3. Determine the number of levels
4. Cost = $\sum_{i=0}^{\hbox{num levels}}$ cost for level $i$
  - This last step usually involves using properties of series
  
<br>

E.g., for merge sort:

- level $i$ contains $2^i$ nodes
- each node at level $i$ costs $c_1 \frac{n}{2^i} + c_2$
- so, each level costs $2^i * (c_1 \frac{n}{2^i} + c_2) = c_1n + 2^i c_2$
- since each level reduces size by half, we have $\lg n$ levels
- so, total cost of tree is:

$$W(n) = \sum_{i=0}^{\lg n} (c_1n + 2^i c_2)$$


> To solve this, we'll make use of bounds for **geometric series**. 
>
> - For $\alpha > 1$: $\:\:\: \sum_{i=0}^n \alpha^i  = \frac{\alpha}{\alpha - 1}\cdot\alpha^n$
> <br>
> 
>  - e.g., $\sum_{i=0}^{\lg n} 2^i < \frac{2}{1} * 2^{\lg n} = 2n$
>
><br>
>
>- For $\alpha < 1$: $\:\:\: \sum_{i=0}^\infty \alpha^i  < \frac{1}{1-\alpha}$
><br>
>
>  - e.g., $\sum_{i=0}^{\lg n} \frac{1}{2^i} < 2$


<br> plugging in...

$$= \sum_{i=0}^{\lg n} (c_1 n + 2^i c_2)$$

$$= \sum_{i=0}^{\lg n}c_1 n + \sum_{i=0}^{\lg n} 2^i c_2$$

$$= c_1n \sum_{i=0}^{\lg n} 1 + c_2 \sum_{i=0}^{\lg n} 2^i$$

$$<c_1n \lg n + 2 c_2 n$$

$$\in O(n \lg n)$$

What about the span?

![alttext](tree.png)


The recurrence for the span of Mergesort is:

$ \begin{equation}
S(n) = \begin{cases}
  c_3, & \text{if $n=1$} \\
  S(n/2) + c_4 \lg n, & \text{otherwise} 
  \end{cases}
\end{equation}$


Since each level of the recursion tree is concurrent and all nodes have the same cost, we have that

$ \begin{align}
S(n) & = & \sum_{i=1}^{\lg n} \lg\frac{n}{2^i}\\
& = & \sum_{i=1}^{\lg n} (\lg n - i)\\
& = & \sum_{i=1}^{\lg n} (\lg n) - \sum_{i=1}^{\lg n} i\\
& < & \lg^2 n  - \frac{1}{2}\lg n * (\lg n+1) \:\: (\hbox{using}\:\:\sum_{i=1}^n = \frac{n(n+1)}{2})\\
& = & \lg^2n - \frac{1}{2}\lg^2 n - \frac{1}{2}  \lg n\\
& = & O(\lg^2 n)\\
\end{align}$


Another recurrence:
    
$ \begin{equation}
W(n) = \begin{cases}
  c_b, & \text{if $n=1$} \\
  2W(n/2) + n^2, & \text{otherwise} 
  \end{cases}
\end{equation}$

What is the asymptotic runtime?

![alttext](tree.png)



$$W(n) = 2W(n/2) + c_1n^2 + c_2$$

$= \sum_i^{\lg n} (c_1 \frac{n^2}{2^i} + 2^i c_2)$

$= c_1 n^2 \sum_{i=0}^{\lg n} \frac{1}{2^i} + c_2 \sum_{i=0}^{\lg n} 2^i$

$< 2 c_1 n^2 + 2 c_2 n$

$\in O(n^2)$

So what if branching factor is not 2?

$$W(n) = 4 W \Big(\frac{n}{2}\Big) + O(n)$$

**costs**

- level 0: $c_1n + c_2$
- level 1: $4(c_1 \frac{n}{2} + c_2)$
- level 2: $16(c_1 \frac{n}{4} + c_2)$
- level $i$ ?

$$4^i(c_1 \frac{n}{2^i} + c_2)$$

<br>

still $\lg n$ levels:, so $W(n)$ is:

<br>

$$= c_1n \sum_{i=0}^{\lg n} \Big(\frac{4}{2}\Big)^i + c_2 \sum_{i=0}^{\lg n} 4^i$$

$$< 2 c_1 n^2 + \frac{4}{3} c_2 4^{\lg n}$$

$$(\hbox{since} \:\:\ \sum_{i=0}^n \alpha^i  < \frac{\alpha}{\alpha - 1}\cdot\alpha^n)$$

$$= 2 c_1 n^2 + \frac{4}{3} c_2 2^{\lg n} 2^{\lg n}$$

$$= 2 c_1 n^2 + \frac{4}{3} c_2 n^2$$

$$\in O(n^2)$$

Start of Lecture 2

Recurrences can be more general, so how do we get a handle on asymptotic runtimes when the recursion is really complicated?

**Key Idea:** The branching properties of the recursion tree determine work at each level and the number of leaves.

The **brick method** gives a way to derive asymptotic runtimes by looking at the relationships between parent and child nodes in the recursion tree. This way we only need to worry about the costs of the root and the leaves.

> In the tree method, if the costs grow or decay geometrically across levels, then we need only consider the cost of the root (**decay**), or the total cost of the leaves (**growth**). 

> If there is no geometric growth or decay then we can calculate the cost of the worst level (often either the root or leaves) and multiply it by the number of levels. 

This leads to three cases:

1. **root dominated**
2. **leaf dominated**
3. **balanced**

> Conveniently, to distinguish these three cases we need only consider the cost of each node in the tree and how it relates to the cost of its children.

The value of $n$ decreases geometrically as we collect the terms in our recurrences. We'll make use of bounds for geometric series. For any $\alpha > 1$:
    
$$ \sum_{i=0}^n \alpha^i  = \frac{\alpha}{\alpha - 1}\cdot\alpha^n$$

For $\alpha < 1$:

$$ \sum_{i=0}^\infty \alpha^i  < \frac{1}{1-\alpha}$$



### Root-dominated

For a node $v$ in the recursion tree, let $C(v)$ denote its cost and $D(v)$ denote its children.

A recurrence is **root-dominated** if for all $v$, there is an $\alpha > 1$ such that:

$$C(v) \geq \alpha \sum_{u \in D(v)} C(u)$$

The cost of a root dominated recurrence is $O(C(r))$ if $r$ is the root.

This is because the cost reduces geometrically as we go toward the leaves, and the total cost bounded by $\alpha/(\alpha-1)$ times $C(r)$.


$$W(n) = 2 W\Big(\frac{n}{2}\Big) + n^2$$

$C(\hbox{root}) = n^2$

$C(\hbox{level}\:1) = \Big(\frac{n}{2}\Big)^2 + \Big(\frac{n}{2}\Big)^2 = \frac{n^2}{2}$

<br>

Cost has **decreased** by a factor of two when descending one level in the tree.

<br>

so, if $\alpha \leftarrow 2$:

$C(v) \ge \alpha \sum_{u \in D(v)} C(u)$

$n^2 \ge 2 * \frac{n^2}{2}$

$n^2 \ge n^2$

<br>

Because the cost is asymptotically dominated by the **root**, we need to only consider its cost: $O(n^2)$.


### Leaf-dominated

A recurrence is **leaf-dominated** if for all $v$, there is an $\alpha > 1$ such that:

$$C(v) \leq \frac{1}{\alpha} \sum_{u \in D(v)} C(u)$$

If we have $L$ leaves in the recursion tree, the cost of a leaf dominated recurrence is $O(L)$.

This is because the cost increases geometrically as we go toward the leaves, and the total cost is bounded by $\alpha/(\alpha-1)$ times $c_b \cdot L$.



$$W(n) = 2W\Big(\frac{n}{2}\Big) + \sqrt{n}$$


$C(\hbox{root}) = \sqrt{n}$

$C(\hbox{level}\:1) = \sqrt{\Big(\frac{n}{2}\Big)} + \sqrt{\Big(\frac{n}{2}\Big)} = 2 \frac{\sqrt{n}}{\sqrt{2}} = \sqrt{2}\sqrt{n}$

<br>

Cost has **increased** by a factor of $\sqrt{2}$ when descending one level in the tree.

<br>

so, if $\alpha \leftarrow \frac{1}{\sqrt{2}}$:

$C(v) \le \frac{1}{\alpha} \sum_{u \in D(v)} C(u)$

$\sqrt{n} \le \frac{1}{\sqrt{2}} * \sqrt{2}\sqrt{n}$

$\sqrt{n} \le \sqrt{n}$

<br>

Because the cost is asymptotically dominated by the **leaves** (i.e., lowest level of the tree), we need to only consider their cost:

<br>

- Cost of each leaf is 1 (since $\sqrt{1}==1$).
- Depth of tree is $\lg{n}$ (since we divide by 2 at each level)
- Number of leaves of a binary tree with depth $\lg{n}$ is $2^{\lg{n}} = n$

<br>
So, final cost is $n*1 = O(n)$


### Balanced

A recurrence is **balanced** when every level of the recursion tree has the same asymptotic cost. In this case, the recurrence is
*number of levels* times *maximum cost per level*.



$O(D(r) \log n) = O(L \log n)$.  


$$W(n) = 2 W\Big(\frac{n}{2}\Big) + n$$

$C(\hbox{root}) = n$

$C(\hbox{level}\:1) = \Big(\frac{n}{2}\Big) + \Big(\frac{n}{2}\Big) = n$

<br>

Cost has **remained the same** when descending one level in the tree.

So, total cost is *number of levels* times *maximum cost per level*.

<br>

- Number of levels = $\lg n$
- Maximum cost per level = $n\:\:$ (last level: $n$ leaves with cost 1 each; first level: one node with cost $n$)
- $O(n \lg n)$


Let's look at some examples:

$$ W(n) = 2 W(n/2) + \sqrt{n} $$

$$ W(n) = 3 W(n/2) + n $$

$$ W(n) = 2 W(n/3) + n $$

$$ W(n) = 3 W(n/3) + n $$

Do you see a way to count the number of leaves in the recursion tree?


>Useful observation: If we have a recurrence of the form $W(n) = aW(n/b) + f(n)$, then the number of leaves is $O(a^{\log_b n})$, or equivalently, $O(n^{log_b a})$.

$$ W(n) = 2 W(n/2) + \sqrt{n} $$

This is leaf-dominated, so it is $O(n)$.

$$ W(n) = 3 W(n/2) + n $$

This is leaf-dominated, so it is $O(n^{\log_2 3})$.

$$ W(n) = 2 W(n/3) + n $$

This is root-dominated, so it is $O(n)$.

$$ W(n) = 3 W(n/3) + n $$

This is balanced, so it is $O(n \log n)$.


More examples (some trickier than others):

$$ W(n) = W(n - 1) + n $$

$$ W(n) = \sqrt{n} W(\sqrt{n}) + n^2 $$

$$ W(n) = W(\sqrt{n}) + W(n/2) + n $$

$$ W(n) = W(n/2) + W(n/3) + 1 $$



$$ W(n) = W(n - 1) + n $$

This is actually a balanced recurrence since every level has the same cost and there are $n$ levels - the recurrence is $O(n^2)$.

$$ W(n) = \sqrt{n} W(\sqrt{n}) + n^2 $$

This is root-dominated so it is $O(n^2)$.

$$ W(n) = W(\sqrt{n}) + W(n/2) + n $$

This is root-dominateed so it is $O(n)$.

$$ W(n) = W(n/2) + W(n/3) + 1 $$

This recurrence is a little tricky - while it is leaf-dominated we need to calculate the number of leaves. The book has a [derivation of this](https://www.diderot.one/courses/152/books/665/chapter/9412#atom-641246).


Start of Lecture 3

### Summary of last lecture

**Brick method:**

- **Root dominated**: cost *decreases* geometrically as we descend the tree
  - $W(n) \in O(\hbox{root})$
  - e.g., $W(n) = 2 W(\frac{n}{2}) + n^2$


- **Leaf dominated**: cost *increases* geometrically as we descend the tree
  - $W(n) \in O(\hbox{leaves})$
  - e.g., $W(n) = 2 W(\frac{n}{2}) + \sqrt{n}$
  
  
- **Balanced**: neither of the above is true
  - $W(n) \in O(\hbox{(num levels)*(max cost at any level)})$
  - e.g., $W(n) = W(n-1) + n \in O(n^2)$
  - e.g., $W(n) = 2 W(\frac{n}{2}) + n \in O(n \lg n)$

### Integer multiplication

Now that we've come up with a general technique for solving recurrences, let's look at a recursive algorithm. You learned this algorithm in elementary school for integer multiplication:

- Input: $n$ bit integers $x= \langle x_{n-1}, \ldots, x_0\rangle$ and $y = \langle y_{n-1}, \ldots, y_0\rangle$

- Output: $x \cdot y$

- Example: '1001'$\times$'1101'


In [5]:
def int2binary(n):
    return list('{0:b}'.format(n))
 
int2binary(9)

['1', '0', '0', '1']

In [6]:
nine = int2binary(9)
print(nine)
thirteen = int2binary(13)
print(thirteen)  

['1', '0', '0', '1']
['1', '1', '0', '1']


```
       1001   
     x 1101   
     ======
       1001   
      0000    
     1001     
  + 1001      
  =========
    1110101   (117)
```

In [7]:
def binary2int(n): 
    return int(n, 2)
binary2int('1110101')

117

What is the running time of the "elementary school" algorithm?

>For two $n$-digit inputs $O(n^2)$, since for each digit of $x$ we might add a stack of $n$ bits. The total number of bits in the solution is at most $2n$.

What does this have to do with recursion and recurrences?

Instead of the elementary school algorithm, consider splitting each $n$-digit input in half. Can we multiply recursively?

Let $x = \langle x_L, x_R\rangle$, $y = \langle y_L, y_R\rangle$. Then,

$\begin{align} 
x &=& 2^{n/2} x_L + x_R \:\:\:\:\:\: \hbox{e.g.} \: 1001:  2^2 (10) + (01)\\
y &=& 2^{n/2} y_L + y_R \:\:\:\:\:\: \hbox{e.g.} \: 1101:  2^2 (11) + (01)\\
\end{align}
$

<br><br>

**Wait...Is multiplying by $2^{n/2}$ efficient?**

<br><br>

Recall: $2^2 [10] \rightarrow [1000] \:\:$ (shift two places to the left).

<br>

So then,

$\begin{align}
x\cdot y &=& (2^{n/2} x_L + x_R)(2^{n/2} y_L + y_R) \\
 &=& 2^n (x_L \cdot y_L) + 2^{n/2} (x_L \cdot y_R + x_R \cdot y_L) + (x_R \cdot y_R) \\
\end{align}
$

<br>

We've converted one multiplication of sizes $(n,n)$ into four multiplications of size $(\frac{n}{2}, \frac{n}{2})$.

What recursive algorithm, and recurrence is suggested by this observation?

>$W(n) = 4W(n/2) + cn$

What is the solution to this recurrence using the brick method? Is it root-dominated, or leaf-dominated?

### work of recursive multiplication

$C(\hbox{root}) = n$

$C(\hbox{level}\:1) = 4(\frac{n}{2})= 2 \cdot n$

geometrically **increasing** as we descend the recurrence tree.

A recurrence is **leaf-dominated** if for all $v$, there is an $\alpha > 1$ such that:

$$C(v) \leq \frac{1}{\alpha} \sum_{u \in D(v)} C(u)$$

let $\alpha \leftarrow 2$

$n \le \frac{1}{2}\cdot 2 \cdot n$

<br>

The cost of a leaf dominated recurrence is $O(L)$, where $L$ is the number of leaves.

### how many leaf nodes are there?

<br><br><br>


nodes per level: $1, 4, 64, \ldots 4^i \ldots 4^{\lg n}=(2\cdot2)^{\lg n}=n\cdot n = n^2$

> This is a leaf-dominated reucrrence that is $O(n^2)$ -- the same as the elementary school algorithm!

Now, what is the span of this algorithm if implemented in parallel?


### span of recursive multiplication

Assuming each multiplication can be done in parallel, and that addition has span $O(n)$, we get that 

$$S(n) = S(n/2) + cn$$ 

which yields a span of ...


<br><br><br><br>

**brick method**

$S(\hbox{root})=n$


$S(\hbox{level 1}) = \frac{n}{2}$

$\rightarrow$ **root dominated** 

$S(n) \in O(n)$. This is much better than the span of the grade school algorithm, which is $O(n^2)$!

<br>

**What parallelism is achieved?**

<br>

Parallelism $= \frac{W}{S} = \frac{n^2}{n} = n$

Can we do any better?

### recursive multiplication with less work

Recall that 

$\begin{align}
x\cdot y &=& (2^{n/2} x_L + x_R)(2^{n/2} y_L + y_R) \\
 &=& 2^n (x_L \cdot y_L) + 2^{n/2} (x_L \cdot y_R + x_R \cdot y_L) + (x_R \cdot y_R) \\
\end{align}
$

<br>

Can we reduce this from 4 multiplications to 3??

<br><br><br><br>

Observation:
    
$\begin{align} 
(x_L + x_R)\cdot (y_L + y_R) &=& (x_L\cdot y_L) + (x_L\cdot y_R) + (x_R\cdot y_L) + (x_R\cdot y_R)\\
\end{align}
$

$\begin{align}
(x_L\cdot y_R) + (x_R\cdot y_L) = (x_L + x_R)\cdot (y_L + y_R) - (x_L\cdot y_L) - (x_R\cdot y_R)\\
\end{align}$




How does our observation help us? 

If we calculate $(x_L\cdot y_L)$, $(x_R\cdot y_R)$, and $(x_L + x_R)\cdot (y_L + y_R)$, that is *three* recursive multiplications. 



So with 3 recursive multiplications and two more "additions", we then get that $W(n) = 3W(n/2) + dn$. What is the running time?

### work of $W(n) = 3W(n/2) + dn$

**brick method**

$C(\hbox{root}) = n$

$C(\hbox{level 1}) = \frac{3n}{2}$


<br><br><br><br>

$\frac{3}{2} > 1 \Rightarrow$ **leaf dominated**.

But, there are fewer leaves this time. Why?

<br><br><br><br>

nodes per level: $1, 3, 9, \ldots 3^i \ldots 3^{\lg n}=n^{\lg 3} \:\:\:\: (\hbox{by}\: a^{\log_b c} = c^{\log_b a})$


<br><br>


Using the brick method, this is still a leaf-dominated recurrence, and thus the running time is $O(n^{\log_2 3}) \:\: $ (approximately $O(n^{1.58}$) versus of $O(n^2)$ earlier).

<br>

This is known as the [**Karatsaba-Ofman**](https://en.wikipedia.org/wiki/Karatsuba_algorithm) algorithm (1962), and is the earliest known divide-and-conquer algorithm for integer multiplication. It is actually implemented in Intel hardware!


<br><br>

So, our we've decreased work from $O(n^2)$ to $O(n^{\log_2 3})$. 

Span stays the same: $O(n)\:\:$  Why?

<br><br>

Parallelism $= \frac{W}{S} = \frac{n^{\log_2 3}}{n} \approx n^{.58} < n$

<br>

So, our parallelism went down. Is that good or bad?

<br><br>


Schönhage and Strassen (1971) gave an algorithm that runs in $O(n\log n \log\log n)$ time.

In 2007, [Fürer gave an algorithm](https://ivv5hpp.uni-muenster.de/u/cl/WS2007-8/mult.pdf) that runs in $n \log n 2^{O(\log^* n)}$.

What is the fastest possible sequential algorithm for integer multiplication? In parallel?
